#  Serve Falcon 40B model with Amazon SageMaker Hosting

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

---

In this example we walk through how to deploy and perform inference on the **Falcon 40B model** using the **Large Model Inference(LMI)** container provided by AWS using **DJL Serving** and **DeepSpeed**. The Falcon 40B model is a casual decoder model that was trained on AWS SageMaker, on 384 A100 40GB GPUs in P4d instances. Because this is a large language model (LLM) that does not fit on a single GPU, we will use an 'ml.g5.24xlarge" instance which has **4** GPUs.


## Setup

Installs the dependencies required to package the model and run inferences using Amazon SageMaker. Update SageMaker, boto3 etc

In [5]:
!pip install --upgrade pip --quiet

In [6]:
!pip install sagemaker boto3 --upgrade  --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [7]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path
from sagemaker.utils import name_from_base

## Imports and variables

In [8]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix_deepspeed = "hf-large-model-djl-/code_falcon40b/deepspeed"  # folder within bucket where code artifact will go

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

### 1. Create SageMaker compatible model artifacts

In order to prepare our model for deployment to a SageMaker Endpoint for hosting, we will need to prepare a few things for SageMaker and our container. We will use a local folder as the location of these files including **serving.properties** that defines parameters for the LMI container and **requirements.txt** to detail what dependies to install.

In [9]:
!mkdir -p code_falcon40b_deepspeed

In [10]:
# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://{bucket}/models/falcon40b/"
print(f"Pretrained model will be downloaded from ---- > {pretrained_model_location}")

Pretrained model will be downloaded from ---- > s3://sagemaker-us-west-2-376678947624/models/falcon40b/


In the **serving.properties** files  define the the **engine** to use and **model** to host. Note the **tensor_parallel_degree** parameter which is also required in this scenario. We will use **tensor parallelism** to divide the model into multiple parts because no single GPU has enough memory for the entire model. In this case we will use a 'ml.g5.24xlarge' instance which provides **4** GPUs. Be careful not to specify a value larger than the instance provides or your deployment will fail. 

In [17]:
%%writefile ./code_falcon40b_deepspeed/serving.properties
engine=DeepSpeed
option.model_id=tiiuae/falcon-40b-instruct
option.tensor_parallel_degree=4
#option.s3url = {{s3url}}

Overwriting ./code_falcon40b_deepspeed/serving.properties


In [18]:
%%writefile ./code_falcon40b_deepspeed/requirements.txt
einops
git+https://github.com/lanking520/DeepSpeed.git@falcon

Overwriting ./code_falcon40b_deepspeed/requirements.txt


### 2. Create a model.py with custom inference code

SageMaker allows you to bring your own script for inference. Here we create our **model.py** file with the appropriate code for the Falcon 40B model.

In [19]:
%%writefile ./code_falcon40b_deepspeed/model.py
from djl_python import Input, Output
import os
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from typing import Any, Dict, Tuple
import deepspeed
import warnings

predictor = None


def get_model(properties):
    model_name = properties["model_id"]
    local_rank = int(os.getenv("LOCAL_RANK", "0"))
    model = AutoModelForCausalLM.from_pretrained(
        model_name, low_cpu_mem_usage=True, trust_remote_code=True, torch_dtype=torch.bfloat16
    )
    model = deepspeed.init_inference(model, mp_size=properties["tensor_parallel_degree"])
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generator = pipeline(
        task="text-generation", model=model, tokenizer=tokenizer, device=local_rank
    )
    return generator


def handle(inputs: Input) -> None:
    global predictor
    if not predictor:
        predictor = get_model(inputs.get_properties())

    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        return None
    data = inputs.get_as_json()
    text = data["text"]
    generation_kwargs = data["properties"]
    outputs = predictor(text, **generation_kwargs)
    result = {"outputs": outputs}
    return Output().add(result)

Overwriting ./code_falcon40b_deepspeed/model.py


### 3. Create the Tarball and then upload to S3 location
Next, we will package our artifacts as `*.tar.gz` files for uploading to S3 for SageMaker to use for deployment

In [20]:
!rm -f model.tar.gz
!rm -rf code_falcon40b_deepspeed/.ipynb_checkpoints
!tar czvf model.tar.gz -C code_falcon40b_deepspeed .
s3_code_artifact_deepspeed = sess.upload_data("model.tar.gz", bucket, s3_code_prefix_deepspeed)
print(f"S3 Code or Model tar for deepspeed uploaded to --- > {s3_code_artifact_deepspeed}")

./
./model.py
./requirements.txt
./serving.properties
S3 Code or Model tar for deepspeed uploaded to --- > s3://sagemaker-us-west-2-376678947624/hf-large-model-djl-/code_falcon40b/deepspeed/model.tar.gz


### 4. Define a serving container, SageMaker Model and SageMaker endpoint
Now that we have uploaded the model artifacts to S3, we can create a SageMaker endpoint.


#### Define the serving container
Here we define the container to use for the model for inference. We will be using SageMaker's Large Model Inference(LMI) container using DeepSpeed. 

In [21]:
# inference_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/djl-ds:latest"
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118


#### Create SageMaker model, endpoint configuration and endpoint.


In [22]:
model_name_ds = name_from_base(f"falcon40b-model-ds")
print(model_name_ds)

falcon40b-model-ds-2023-07-27-01-43-03-335


In [23]:
create_model_response = sm_client.create_model(
    ModelName=model_name_ds,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": s3_code_artifact_deepspeed},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

Created Model: arn:aws:sagemaker:us-west-2:376678947624:model/falcon40b-model-ds-2023-07-27-01-43-03-335


In [24]:
model_name = model_name_ds
print(f"Building EndpointConfig and Endpoint for: {model_name}")

Building EndpointConfig and Endpoint for: falcon40b-model-ds-2023-07-27-01-43-03-335


In [25]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.24xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
            # "VolumeSizeInGB": 512
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:376678947624:endpoint-config/falcon40b-model-ds-2023-07-27-01-43-03-335-config',
 'ResponseMetadata': {'RequestId': '152cea0a-4847-436d-ac1d-21d054fd9341',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '152cea0a-4847-436d-ac1d-21d054fd9341',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '130',
   'date': 'Thu, 27 Jul 2023 01:43:04 GMT'},
  'RetryAttempts': 0}}

In [26]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:376678947624:endpoint/falcon40b-model-ds-2023-07-27-01-43-03-335-endpoint


In [27]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:376678947624:endpoint/falcon40b-model-ds-2023-07-27-01-43-03-335-endpoint
Status: InService


### Run Inference

Large models such as Falcon have very high accelerator memory footprint. Thus, a very large input payload or generating a large output can cause out of memory errors. The inference examples below are calibrated such that they will work on the `ml.g5.24xlarge` instance within the
SageMaker response time limit of 60 seconds. If you find that increasing the input length or generation length leads to CUDA Out Of Memory errors, we recommend that you try one of the following solutions:
* Use 8 bit quantization. In the `model.py` script, you can enable `dtype=torch.int8` in the call to `deepspeed.init_inference`. This will reduce the memory footprint of the model on the GPUs, allowing for larger input and generation sizes.
  * Using 8bit quantization may result in lower quality generated output.
* Deploy to an instance with more GPUs, and/or GPUs with more memory. The `ml.g5.48xlarge`, `ml.p4d.24xlarge`, and `ml.p4de.24xlarge` instances are all good options here.

When attempting to generate more tokens, you might run into issues with the SageMaker runtime client timing out after 60 seconds. To get around this issue, we recommend that you check out our example for Large Language Models with streaming via pagination.
You can find that example [here](../lab6-stream-with-pagination/stream_pagination_lmi.ipynb). When using streaming, you still have to be conscious of memory constraints.

In the following example inference requests, we limit the sequence length such that we return a response within 60 seconds and don't exceed the GPU memory available.

You can pass additional generation arguments as part of the properties dictionary in the request (e.g. temperature, top_k etc.).

In [31]:
%%time

data = {
    "text": "What is the purpose of life?",
    "properties": {
        "min_length": 100,
        "max_length": 150,
        "do_sample": True,
    },
}

response_model = smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(data),
    ContentType="application/json",
)

response_model["Body"].read().decode("utf8")

CPU times: user 16.8 ms, sys: 609 µs, total: 17.4 ms
Wall time: 21.3 s


'{\n  "outputs":[\n    {\n      "generated_text":"What is the purpose of life?\\nAs an AI language model, I am not capable of inferring the meaning of life. The purpose of life is subjective and varies depending on personal beliefs, values and experiences. Some argue that the purpose of life is to find happiness, fulfillment, to contribute to the greater good, or to explore the universe. However, ultimately the purpose of life remains a mystery to be unraveled. \\"Is there a purpose in life?\\" is a philosophical question that has been pondered by humans since ancient times. Many people believe that the answer is \\"yes,\\" while others think that it is \\"no.\\" Perhaps, in due course, we will come upon an answer that makes sense to us."\n    }\n  ]\n}'

In [29]:
start_time = time.time()

data = {
    "text": "What is Love?",
    "properties": {
        "min_length": 100,
        "max_length": 150,
        "do_sample": True,
    },
}

while (time.time() - start_time) < 300:  # 300 seconds = 5 minutes
    response_model = smr_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(data),
        ContentType="application/json",
    )

    print("Loop restarting - answer: " + response_model["Body"].read().decode("utf8"))

Loop restarting - answer: {
  "outputs":[
    {
      "generated_text":"What is Love?\nLove is an emotional and spiritual bond that binds people together. It is a feeling of intense affection, warmth or deep attraction for someone or something. Love can exist in many forms, such as the love between family members, romantic partners, friends, and even pets. It is often associated with caring, kindness, and compassion. Love can be expressed in many ways, such as through words, actions, and gestures. And when love is truly present, it can transform a person's life and enhance their sense of joy and happiness."
    }
  ]
}
Loop restarting - answer: {
  "outputs":[
    {
      "generated_text":"What is Love?\nLove is an emotion that is known for its ability to transform a person. It is a feeling that is characterized by the desire to give to someone, the desire to have that person close to oneself, and a willingness to do anything for them. It's important to remember that love doesn't happe

### Clean Up

In [ ]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/inference%7Cgenerativeai%7Cllm-workshop%7Clab10-falcon-40b-and-7b%7Cfalcon-40b-deepspeed.ipynb)
